In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,5'
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import json
from constants_prompt import build_autoj_input

model_path_list = ['/root/Qwen1.5-7B-Chat', '/root/Meta-Llama-3-8B-Instruct', '/root/.cache/modelscope/hub/qwen/Qwen1___5-14B-Chat',
                   '/root/Nanbeige2-8B-Chat', '/root/.cache/modelscope/hub/ZhipuAI/chatglm3-6b', 
                   '/root/.cache/modelscope/hub/Shanghai_AI_Laboratory/internlm2-chat-7b',
                   '/root/.cache/modelscope/hub/deepseek-ai/deepseek-llm-7b-chat',
                   '/root/.cache/modelscope/hub/lockonlvange/autoj-13b-fp16',
                   '/root/.cache/huggingface/hub/models--WeOpenML--PandaLM-7B-v1/snapshots/PandaLM',
                   '/root/finetuned/qwen1.5-7b-chat_autoj_trainset_qlora_epoch_5']

In [2]:
model_name_or_path = model_path_list[-1]
llm = LLM(model=model_name_or_path, tensor_parallel_size=4, trust_remote_code=True, tokenizer_mode="auto", dtype="auto", gpu_memory_utilization=0.85)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
sampling_params = SamplingParams(temperature=0.0, top_p=1.0, max_tokens=1024)
model_name = model_name_or_path.rsplit('/', 1)[-1]
from utils import extract_pariwise_result

2024-06-06 07:07:47,296	INFO worker.py:1749 -- Started a local Ray instance.


INFO 06-06 07:07:48 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='/root/finetuned/qwen1.5-7b-chat_autoj_trainset_qlora_epoch_5', speculative_config=None, tokenizer='/root/finetuned/qwen1.5-7b-chat_autoj_trainset_qlora_epoch_5', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=4, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/root/finetuned/qwen1.5-7b-chat_autoj_trainset_qlora_epoch_5)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-06 07:07:58 utils.py:618] Found nccl from library libnccl.so.2
INFO 06-06 07:07:58 pynccl.py:65] vLLM is using nccl==2.20.5
(RayWorkerWrapper pid=2067986) INFO 06-06 07:07:58 utils.py:618] Found nccl from library libnccl.so.2
(RayWorkerWrapper pid=2067986) INFO 06-06 07:07:58 pynccl.py:65] vLLM is using nccl==2.20.5
WARNING 06-06 07:07:58 custom_all_reduce.py:158] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
(RayWorkerWrapper pid=2067986) WARNING 06-06 07:07:58 custom_all_reduce.py:158] Custom allreduce is disabled because it's not supported on more than two PCIe-only GPUs. To silence this warning, specify disable_custom_all_reduce=True explicitly.
INFO 06-06 07:08:00 model_runner.py:146] Loading model weights took 3.6506 GB
(RayWorkerWrapper pid=2067986) INFO 06-06 07:08:00 model_runner.py:146] Loading model weights took 3.6506 GB
INFO 06-06 07:08:05 distribu

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# autoj
texts_autoj = []
input_file_path = '../datasets/AutoJ/testdata_pairwise.jsonl'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_autoj_input(prompt=item['prompt'], resp1=item['response 1'], resp2=item['response 2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_autoj.append(text)
outputs_autoj = llm.generate(texts_autoj, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_autoj, fin):
            item = json.loads(line)
            pred_label = extract_pariwise_result(output.outputs[0].text)
            item['response_judge'] = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)

Processed prompts:   0%|          | 0/1392 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

WARNING 06-06 07:08:33 scheduler.py:1077] Sequence group 110 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts: 100%|██████████| 1392/1392 [07:31<00:00,  3.08it/s, Generation Speed: 848.01 toks/s]

0
0
0
1
0
0
1
1
0
0
0
0
1
1
0
0
0
0
1
1
0
1
0
0
0
1
1
0
1
1
0
0
1
1
1
1
1
1
1
0
1
0
0
1
1
1
1
0
1
1
1
1
0
1
0
1
0
0
0
0
0
1
1
0
1
0
0
0
0
0
0
0
1
0
0
0
0
1
1
0
0
1
0
1
0
0
1
1
0
0
1
0
0
0
1
1
1
0
0
1
0
0
1
0
0
1
1
0
0
0
0
0
0
1
1
1
1
1
0
1
0
1
1
0
0
1
1
0
0
0
1
0
1
0
1
0
0
1
0
0
0
1
0
0
0
1
0
1
1
1
1
1
0
0
0
0
0
0
1
0
1
1
0
0
1
1
1
1
1
0
0
0
1
0
1
1
0
0
0
1
0
1
0
0
1
0
1
0
1
1
0
0
0
1
0
0
1
0
0
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
1
0
1
0
0
1
0
1
0
0
1
0
0
1
0
1
1
1
0
1
0
1
0
0
0
1
0
0
1
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
1
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
1
0
1
0
1
2
2
0
0
1
0
0
1
1
1
0
1
0
1
1
1
0
0
0
1
0
0
0
1
0
0
0
0
1
0
0
0
1
2
1
2
1
1
0
1
0
0
1
0
0
1
0
1
0
1
1
0
0
0
0
0
0
1
0
0
0
0
1
0
1
1
0
1
0
1
0
0
1
0
0
0
1
0
1
0
0
1
0
0
1
1
0
0
1
1
0
0
1
0
0
0
0
0
0
1
0
0
1
0
1
1
0
0
1
1
1
1
1
0
0
0
0
0
1
0
0
1
1
1
1
0
0
1
1
1
1
1
0
1
0
1
1
1
0
0
0
1
1
0
1
0
1
1
1
0
0
1
1
0
0
0
1
1
1
1
1
0
1
1
1
1
1
0
1
1
1
1
0
1
1
1
0
1
0
1
0
0
0
1
1
0
1
1
1
1
0
0
0
0
0
0
1
1
0
1
0
0
1
1
1
0
0
0
0
0
1
0
1
0
0


In [4]:
# pandalm
prompt_templates = {
    "alpaca": {
        "description": "Template used by Alpaca-LoRA.",
        "prompt_input": "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
        "prompt_no_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n",
        "response_split": "### Response:",
    }
}
def get_prompt(instruction, input):
    prompt = prompt_templates['alpaca']["prompt_input"].format(instruction=instruction, input=input)
    return prompt


texts_pandalm = []
    
import json
input_file_path = '../datasets/PandaLM/testset-v1.json'
with open(input_file_path, 'r') as f:
    input_data_pandalm = json.load(f)
    for item in input_data_pandalm:
        prompt = build_autoj_input(prompt=get_prompt(item['instruction'], item['input']), resp1=item['response1'], resp2=item['response2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_pandalm.append(text)

outputs_pandalm = llm.generate(texts_pandalm, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_pandalm, input_data_pandalm):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts:   0%|          | 0/999 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

WARNING 06-06 07:16:10 scheduler.py:1077] Sequence group 1615 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=51


Processed prompts: 100%|██████████| 999/999 [03:07<00:00,  5.34it/s, Generation Speed: 1182.19 toks/s]

0
0
0
0
1
1
0
0
2
0
0
0
0
1
1
0
0
0
1
1
1
1
1
1
1
1
0
0
0
1
1
0
2
1
1
1
0
1
1
0
1
0
0
1
1
1
0
1
0
1
1
1
0
0
0
1
0
1
0
0
0
0
0
1
0
0
0
0
1
0
1
0
1
0
0
0
1
0
1
1
0
1
1
1
0
0
1
1
0
0
2
2
2
2
2
2
2
2
0
0
1
1
0
1
1
2
1
0
0
0
1
0
0
1
2
1
2
0
1
1
2
1
1
2
1
1
1
1
0
1
1
0
0
1
1
0
1
0
0
1
2
2
0
2
0
0
1
0
0
0
1
0
0
0
0
1
1
2
2
2
2
2
2
2
2
0
0
0
1
1
1
1
1
1
1
0
1
0
1
1
0
0
1
0
1
0
0
0
0
0
0
0
1
0
0
1
1
1
0
0
0
0
0
1
2
2
1
1
1
0
1
0
0
0
1
1
2
1
1
1
1
0
0
1
1
1
2
1
1
2
2
2
2
0
0
0
1
2
0
0
2
1
1
1
2
1
1
0
0
0
0
0
1
0
0
1
0
0
0
0
0
0
0
0
0
2
0
0
0
0
1
0
1
1
0
1
0
1
0
2
0
1
1
0
0
0
0
0
0
2
1
0
0
1
0
0
0
0
1
1
0
0
2
0
0
0
1
1
0
0
1
0
0
0
0
0
0
1
0
1
1
0
0
0
0
0
0
0
1
1
0
1
0
1
1
1
1
1
0
0
1
2
1
1
0
0
1
1
0
1
1
0
1
1
1
1
1
1
2
2
2
2
2
2
2
0
1
1
1
1
1
1
0
0
0
2
0
0
2
2
2
2
2
0
0
1
1
0
1
0
0
0
0
0
0
0
0
0
1
2
2
2
2
2
2
1
1
1
0
0
1
1
1
0
0
0
0
0
0
0
0
2
1
0
0
0
2
1
0
1
1
0
0
0
0
0
0
1
1
0
1
1
1
1
0
0
1
0
1
1
0
1
1
1
0
0
0
0
0
1
0
2
2
0
2
0
2
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
1
1
1
0
0
0
2
2
2
2
0
0
0
0
0
1
1
0


In [5]:
# llmbar_natural
texts_natural = []
input_file_path = '../datasets/LLMBar/Natural/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_natural = json.load(f)
    for item in input_data_llmbar_natural:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_natural.append(text)
outputs_llmbar_natural = llm.generate(texts_natural, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_natural, input_data_llmbar_natural):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 100/100 [00:20<00:00,  4.88it/s, Generation Speed: 1129.43 toks/s]

0
0
1
1
1
0
0
0
1
0
0
1
0
1
0
1
1
1
1
1
1
1
0
0
2
1
0
0
1
1
0
0
1
1
0
0
0
0
0
1
0
0
0
1
1
1
0
0
1
1
1
1
0
1
1
0
1
1
0
1
1
0
0
1
2
1
0
0
0
0
0
1
2
0
1
1
1
1
1
0
1
1
0
0
1
2
1
0
1
0
1
0
1
1
1
0
1
1
1
1


In [6]:
# llmbar_neighbor

texts_neighbor = []
input_file_path = '../datasets/LLMBar/Adversarial/Neighbor/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_neighbor = json.load(f)
    for item in input_data_llmbar_neighbor:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_neighbor.append(text)
outputs_llmbar_neighbor = llm.generate(texts_neighbor, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_neighbor, input_data_llmbar_neighbor):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 134/134 [00:26<00:00,  5.08it/s, Generation Speed: 1235.88 toks/s]

0
0
2
1
1
0
0
0
0
1
1
0
1
0
0
0
0
1
1
0
0
1
0
0
0
1
0
2
0
1
0
1
0
1
0
1
0
1
1
1
0
1
1
0
1
1
0
0
1
0
0
1
0
1
0
0
2
1
1
0
1
1
1
0
1
1
0
0
1
1
1
2
1
0
0
1
0
0
2
2
0
0
0
0
1
0
0
1
1
1
0
1
1
1
1
1
0
1
0
1
1
1
0
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
1
0
1
0
1
1
1
0
1
0
0
0
0
1
1


In [7]:
# llmbar_gptinst

texts_gptinst = []
input_file_path = '../datasets/LLMBar/Adversarial/GPTInst/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_gptinst = json.load(f)
    for item in input_data_llmbar_gptinst:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_gptinst.append(text)
outputs_llmbar_gptinst = llm.generate(texts_gptinst, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_gptinst, input_data_llmbar_gptinst):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 92/92 [00:27<00:00,  3.31it/s, Generation Speed: 977.08 toks/s] 

1
0
1
0
0
0
0
0
0
0
1
0
1
0
0
1
0
1
1
0
0
1
0
0
0
1
0
0
0
1
1
0
1
0
0
1
0
0
1
1
0
1
1
1
1
1
0
0
1
1
0
1
0
1
1
0
1
0
0
0
0
1
0
1
0
0
1
1
0
1
1
1
1
0
0
0
0
0
1
0
0
1
1
0
1
1
1
1
1
0
0
0


In [8]:
# llmbar_gptout

texts_gptout = []
input_file_path = '../datasets/LLMBar/Adversarial/GPTOut/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_gptout = json.load(f)
    for item in input_data_llmbar_gptout:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_gptout.append(text)
outputs_llmbar_gptout = llm.generate(texts_gptout, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_gptout, input_data_llmbar_gptout):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 47/47 [00:11<00:00,  3.99it/s, Generation Speed: 997.95 toks/s]

1
0
0
1
1
0
1
1
0
0
0
0
2
1
1
2
0
0
1
1
1
0
1
0
0
1
0
2
0
0
1
0
0
0
0
0
1
0
1
0
1
0
0
0
1
0
0


In [9]:
# llmbar_manual

texts_manual = []
input_file_path = '../datasets/LLMBar/Adversarial/Manual/dataset.json'
with open(input_file_path, 'r') as f:
    input_data_llmbar_manual = json.load(f)
    for item in input_data_llmbar_manual:
        prompt = build_autoj_input(prompt=item['input'], resp1=item['output_1'], resp2=item['output_2'], protocol="pairwise_tie")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_manual.append(text)
outputs_llmbar_manual = llm.generate(texts_manual, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response.json'
with open(output_file_path, 'w') as f:
    for output, item in zip(outputs_llmbar_manual, input_data_llmbar_manual):
        pred_label = extract_pariwise_result(output.outputs[0].text)
        item['response_judge'] = output.outputs[0].text
        item['pred_label'] = pred_label
        json.dump(item, f)
        f.write('\n')
        print(pred_label)

Processed prompts: 100%|██████████| 46/46 [00:13<00:00,  3.31it/s, Generation Speed: 851.88 toks/s]

0
0
1
0
1
1
0
0
0
1
0
0
1
1
0
0
0
0
1
0
1
1
0
1
2
1
0
0
0
1
0
1
0
0
0
1
1
0
0
0
0
1
1
0
0
0


In [10]:
# mt-bench-type1
import json
texts_mtbench = []
input_file_path = '../datasets/MTBench/mt_bench_human.json'
with open(input_file_path, 'r') as f:
    for line in f:
        item = json.loads(line)
        prompt = build_autoj_input(prompt=item['conversation_a'][0]['content'],
                                   resp1=item['conversation_a'][1]['content'],
                                   resp2=item['conversation_b'][1]['content'],
                                   prompt2=item['conversation_a'][2]['content'],
                                   resp1_2=item['conversation_a'][3]['content'],
                                   resp2_2=item['conversation_b'][3]['content'],
                                   protocol="multiturn_pairwise_tie_type1")
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        texts_mtbench.append(text)
outputs_mtbench = llm.generate(texts_mtbench, sampling_params)
dataset_name = input_file_path.rsplit('/', 1)[-1].split('.')[0]
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/'
if not os.path.exists(output_file_path):
    os.mkdir(output_file_path)
output_file_path = input_file_path.rsplit('/', 1)[0] + '/' + model_name + '/' + dataset_name + '_response_type1.json'
with open(input_file_path, 'r') as fin:
    with open(output_file_path, 'w') as f:
        for output, line in zip(outputs_mtbench, fin):
            item = json.loads(line)
            pred_label = extract_pariwise_result(output.outputs[0].text)
            item['response_judge'] = output.outputs[0].text
            item['pred_label'] = pred_label
            json.dump(item, f)
            f.write('\n')
            print(pred_label)

Processed prompts:   0%|          | 0/3355 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

WARNING 06-06 07:21:11 scheduler.py:1077] Sequence group 2892 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=101


Processed prompts:  16%|█▌        | 530/3355 [03:38<27:48,  1.69it/s, Generation Speed: 755.16 toks/s]

WARNING 06-06 07:24:29 scheduler.py:1077] Sequence group 3439 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=151


Processed prompts:  43%|████▎     | 1448/3355 [08:18<11:07,  2.86it/s, Generation Speed: 820.99 toks/s] 

WARNING 06-06 07:29:09 scheduler.py:1077] Sequence group 4372 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=201


Processed prompts:  84%|████████▍ | 2827/3355 [17:48<04:00,  2.20it/s, Generation Speed: 751.80 toks/s]

WARNING 06-06 07:38:39 scheduler.py:1077] Sequence group 5709 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=251


Processed prompts: 100%|██████████| 3355/3355 [22:25<00:00,  2.49it/s, Generation Speed: 736.10 toks/s]


1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
1
0
0
1
1
1
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
1
1
0
0
1
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
1
0
0
0
0
0
0
1
1
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
0
0
1
1
0
0
0
0
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
1
1
1
1
0
0
1
1
1
1
0
0
